In [54]:
import googlemaps
from datetime import datetime
import random
import folium
import math


def generate_points(destination, num_points, radius_km):
    """
    Generates random points around a destination within a specified radius.

    Args:
        destination (tuple): A tuple containing the destination's latitude and longitude, e.g., (lat, lng).
        num_points (int): The number of random points to generate.
        radius_km (float): The radius (in kilometers) around the destination within which points will be generated.

    Returns:
        list: A list of tuples, each containing the latitude and longitude of a generated point.
    """
    generated_points = []

    for _ in range(num_points):
        # Generate random offsets for latitude and longitude within the specified radius
        lat_offset_km = random.uniform(-radius_km, radius_km)
        lng_offset_km = random.uniform(-radius_km, radius_km)

        # Calculate the new point's coordinates
        new_lat = destination[0] + (
            lat_offset_km / 111.32
        )  # Approximate conversion from km to degrees
        new_lng = destination[1] + (
            lng_offset_km / (40075 / 360.0)
        )  # Approximate conversion from km to degrees

        generated_points.append((new_lat, new_lng))

    return generated_points


def distribute_points_in_circle(center, num_points, radius):
    generated_points = []
    for i in range(num_points):
        angle = (2 * math.pi / num_points) * i
        new_lat = center[0] + (radius / 111.32) * math.cos(angle)
        new_lng = center[1] + (radius / (40075 / 360.0)) * math.sin(angle)
        generated_points.append((new_lat, new_lng))
    return generated_points


# Example usage:
destination = (51.516561, -0.134982)  # Example coordinates for New York City
num_points = 25
radius_km = 3
generated_points = distribute_points_in_circle(destination, num_points, radius_km)
# + generate_points(destination, num_points, radius_km)

# Create a map centered at the destination
m = folium.Map(location=destination, zoom_start=12)

# Add the destination marker with a different color
destination_color = "green"  # Change the color as needed
folium.Marker(destination, icon=folium.Icon(color=destination_color)).add_to(m)

# Add markers for each generated point
for point in generated_points:
    folium.Marker(point).add_to(m)

# Display the map in the IPython notebook
m

In [ ]:
def get_driving_duration(result_json):
    try:
        return result_json[0]['legs'][0]['duration']['value']
    except:
        return 999


gmaps = googlemaps.Client(key="yourkeyhere")

# Request directions via driving
now = datetime.now()
compiled_results = []
for source_pt in generated_points:
    directions_result = gmaps.directions(
        source_pt, destination, departure_time=now
    )
    driving_duration = get_driving_duration(directions_result)
    compiled_results.append(directions_result)
    # break

In [37]:
import json

# Specify the file path where you want to save the content
file_path = "directions_response.json"

# Write the content to a file
with open(file_path, "w") as file:
    json.dump(compiled_results, file, indent=4)

In [53]:
import pandas as pd


def parse_directions_response(data):
    # Extract relevant information
    route = data[0]  # Assuming there's only one route

    # Extract information from the 'legs' section of the route
    legs = route["legs"][0]
    distance_all = legs["distance"]["text"]
    duration_all = legs["duration"]["text"]
    duration_sec_all = legs["duration"]["value"]
    start_address = legs["start_address"]
    end_address = legs["end_address"]

    step_data = []
    # Extract information from each step in the 'steps' section of the leg
    steps = legs["steps"]
    for step in steps:
        distance = step["distance"]["text"]
        duration = step["duration"]["text"]
        instructions = step["html_instructions"]
        step_data.append(
            {
                "Distance": distance,
                "Duration": duration,
                "Instructions": instructions,
            }
        )

    # Create a Pandas DataFrame
    df = pd.DataFrame(step_data)

    # Add additional columns for route-level information
    df["Total Distance"] = distance_all
    df["Total Duration"] = duration_all
    df["Total Duration Seconds"] = duration_sec_all
    df["Start Address"] = start_address
    df["End Address"] = end_address

    return df


# Example usage:
response_data = compiled_results[0]  # Replace with the JSON data from the API response
df = parse_directions_response(response_data)

# Display the DataFrame
df

,Distance,Duration,Instructions,Total Distance,Total Duration,Total Duration Seconds,Start Address,End Address
0,0.1 km,1 min,Head <b>southwest</b> on <b>Gower Pl</b> towar...,1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
1,0.4 km,2 mins,Turn <b>right</b> onto <b>Grafton Way</b><div ...,1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
2,0.2 km,1 min,<b>Grafton Way</b> turns <b>left</b> and becom...,1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
3,0.3 km,1 min,"Continue onto <b>Charlotte St.</b><div style=""...",1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
4,0.1 km,1 min,Turn <b>right</b> onto <b>Goodge St</b>/<wbr/>...,1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
5,0.3 km,1 min,Turn <b>left</b> onto <b>Newman St</b>,1.3 km,6 mins,364,"215 Euston Rd., London WC1E 6BP, UK","9-10 Newman St, London W1T 1PB, UK"
